In [1]:
import os
import exifread
from decimal import Decimal
from position_utils import *
import requests
import json
import datetime

In [2]:
class Location(object):

    def __init__(self, image_path):
        self.img_path = image_path

        self.api_key = "你申请的AK"

        self.url_get_position = 'https://restapi.amap.com/v3/geocode/regeo?key={}&location={}'

    def run(self):
        coordinate = self.__get_image_ability()

        print(f'获取到经度、纬度是:{coordinate}')

        if not coordinate:
            return

        # 根据经度和纬度，获取到详细地址
        address = self.__get_address(coordinate)

        # 检验坐标值
        # https://lbs.amap.com/console/show/picker
        print(f'你女朋友当前位置在:{address}')

    def __get_address(self, location):
        """
        根据坐标得到详细地址
        :param location: 经纬度值
        :return:
        """
        resp = requests.get(self.url_get_position.format(self.api_key, location))

        location_data = json.loads(resp.text)

        address = location_data.get('regeocode').get('formatted_address')

        return address

    def __format_lati_long_data(self, data):
        """
        对经度和纬度数据做处理，保留6位小数
        :param data: 原始经度和纬度值
        :return:
        """
        # 删除左右括号和空格
        data_list_tmp = str(data).replace('[', '').replace(']', '').split(',')
        data_list = [data.strip() for data in data_list_tmp]

        # 替换秒的值
        data_tmp = data_list[-1].split('/')

        # 秒的值
        data_sec = int(data_tmp[0]) / int(data_tmp[1]) / 3600

        # 替换分的值
        data_tmp = data_list[-2]

        # 分的值
        data_minute = int(data_tmp) / 60

        # 度的值
        data_degree = int(data_list[0])

        # 由于高德API只能识别到小数点后的6位
        # 需要转换为浮点数，并保留为6位小数
        result = "%.6f" % (data_degree + data_minute + data_sec)
        return float(result)

    def __get_image_ability(self):
        """
        获取图片的属性值，包含：经纬度、拍摄时间等
        :param picture_name:
        :return:
        """

        # 利用exifread库，读取图片的属性
        img_exif = exifread.process_file(open(self.img_path, 'rb'))

        # 能够读取到属性
        if img_exif:
            # 纬度数
            latitude_gps = img_exif['GPS GPSLatitude']

            # N,S 南北纬方向
            latitude_direction = img_exif['GPS GPSLatitudeRef']

            # 经度数
            longitude_gps = img_exif['GPS GPSLongitude']

            # E,W 东西经方向
            longitude_direction = img_exif['GPS GPSLongitudeRef']

            # 拍摄时间
            take_time = img_exif['EXIF DateTimeOriginal']

            is_lie = self.judge_time_met(take_time)

            if is_lie:
                print('很遗憾的通知你，你的女朋友在撒谎！！！')
                return

                # 纬度、经度、拍摄时间
            if latitude_gps and longitude_gps and take_time:

                # 对纬度、经度值原始值作进一步的处理
                latitude = self.__format_lati_long_data(latitude_gps)
                longitude = self.__format_lati_long_data(longitude_gps)

                # print(f'{longitude},{latitude}')

                # 注意：由于gps获取的坐标在国内高德等主流地图上逆编码不够精确，这里需要转换为火星坐标系
                location = wgs84togcj02(longitude, latitude)

                return f'{location[0]},{location[1]}'
            else:
                print(f'获取的图片数据属性不完整')
                return ''
        else:
            print('抱歉，图片不是原图，没法获取到图片属性。')
            return ''

    def judge_time_met(self, take_time):
        """
        通知拍摄时间判断女朋友是否撒谎
        :param take_time:
        :return:
        """
        # 拍摄时间
        format_time = str(take_time).split(" ")[0].replace(":", "-")

        # 当天日期
        today = str(datetime.date.today())

        if format_time == today:
            return False
        else:
            return True

In [3]:
import exifread

In [5]:
img_exif = exifread.process_file(open(r'D:\777.jpg', 'rb'))# 利用exifread库读取图片属性
#img_exif = exifread.process_file(open(r'D:\876.jpg', 'rb'))# 利用exifread库读取图片属性
img_exif

{'Image Orientation': (0x0112) Short=Horizontal (normal) @ 18,
 'Image XResolution': (0x011A) Ratio=72 @ 86,
 'Image YResolution': (0x011B) Ratio=72 @ 94,
 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 54,
 'Image YCbCrPositioning': (0x0213) Short=Centered @ 66,
 'Image ExifOffset': (0x8769) Long=102 @ 78,
 'Thumbnail Compression': (0x0103) Short=JPEG (old-style) @ 202,
 'Thumbnail XResolution': (0x011A) Ratio=72 @ 270,
 'Thumbnail YResolution': (0x011B) Ratio=72 @ 278,
 'Thumbnail ResolutionUnit': (0x0128) Short=Pixels/Inch @ 238,
 'Thumbnail JPEGInterchangeFormat': (0x0201) Long=286 @ 250,
 'Thumbnail JPEGInterchangeFormatLength': (0x0202) Long=10615 @ 262,
 'EXIF ExifVersion': (0x9000) Undefined=0221 @ 112,
 'EXIF ComponentsConfiguration': (0x9101) Undefined=YCbCr @ 124,
 'EXIF FlashPixVersion': (0xA000) Undefined=0100 @ 136,
 'EXIF ColorSpace': (0xA001) Short=sRGB @ 148,
 'EXIF ExifImageWidth': (0xA002) Long=4032 @ 160,
 'EXIF ExifImageLength': (0xA003) Long=3024 @ 172,
 'EX